In [28]:
import pandas as pd
import polars as pl
import os
from tqdm import tqdm

In [29]:
folder = '.\\input_files\\'
file = '231352213_StLukesHospitalAllentownCampus_standardcharges.csv'

df = pd.read_csv(folder + file, dtype=str)

In [30]:
df.rename(columns={
    'Record ID': 'local_code',
    'Description': 'description',
    'CPT/DRG': 'code',
}, inplace=True)

In [31]:
cols = df.columns.tolist()
id_vars = cols[:3]
value_vars = cols[3:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='standard_charge')

In [32]:
df.loc[df['code'].str.len() == 3, 'ms_drg'] = df['code']
df.loc[df['code'].str.len() == 5, 'hcpcs_cpt'] = df['code']

In [33]:
df['payer'] = df['payer'].str.strip()

In [34]:
mapping = {
    'Gross Charge': 'gross',
    'Self Pay': 'cash',
    'Min Payment': 'min',
    'Max Payment': 'max'
}

df['payer_category'] = df['payer'].map(mapping).fillna('payer')

In [40]:
df['standard_charge'] = df['standard_charge'].str.replace('\$|,|\(|\)', '', regex=True)

In [41]:
df['standard_charge'] = df['standard_charge'].astype(float)

In [50]:
df['ms_drg'].fillna('', inplace=True)
df['hcpcs_cpt'].fillna('', inplace=True)
df['code'].fillna('', inplace=True)
df['local_code'].fillna('', inplace=True)

In [51]:
ccn = {
    '844475996': '390162',
    '824432109': '390332',
    '465143606': '390330',
    '454394739': '390326',
    '251550350': '390183',
    '231352213': '390049',
    '231352213': '390335',
    '231352203': '390035',
    '221494454': '310060'
}

ein = file.split('_')[0]

df['hospital_id'] = ccn[ein]

In [54]:
df1 = pl.from_pandas(df)
df1.write_csv('.\\output_files\\' + file.split('_')[1] + '.csv')